20181114

# Chapter 5 誤差反向傳播法
1. 計算圖 computational graph
2. 算式

計算圖的idea Andrej Karpathy and Fei-Fei Li  CS231n
計算圖上由左往右稱之為forward propagation, 反向則稱之為backward propagation

計算圖真正的用意就是，
1. 其實每個局部計算的結果其實是不需要依賴前面的結果就能計算自身節點的產出。
2. 每個節點如果需要前面數字，也可以直接參考引用，並不需要再重新計算一次


# 乘法層

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x * y
        return out
    def backward(self,dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [4]:
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

220.00000000000003


In [5]:
# backward
dprice = 1
dapple_price,dtax = mul_tax_layer.backward(dprice)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)
print (dapple,dapple_num,dtax)

2.2 110.00000000000001 200


## 加法層

In [9]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self,x,y):
        return x+y
    def backward(self,dout):
        dx = dout*1
        dy = dout*1
        return dx,dy


In [11]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_range_layer = AddLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple,apple_num)
orange_price = mul_orange_layer.forward(orange,orange_num)
all_price = add_apple_range_layer.forward(apple_price,orange_price)
price = mul_tax_layer.forward(all_price,tax)

print(price)

#backward
dprice = 1
dall_price,dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_range_layer.backward(dall_price)
dorange,dorange_num = mul_orange_layer.backward(dorange_price)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple_num,dapple,dorange_num,dorange,dtax)

715.0000000000001
110.00000000000001 2.2 165.0 3.3000000000000003 650


## 建立其他函數的層
### ReLU layer

In [12]:
# 類似電源開關，要是一開始正向傳播有過去的話，表示這個電路是通的，那回來的時候也就是表示原路通過。反之不通就是不通
class Relu:
    def __init___(self):
        self.mask = None
    def forward(self,x):
        #只要是小於 0 的就要準備讓她都是0
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dcout
        return dx

### sigmoid layer

In [14]:
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self,x):
        out = 1 / (1+np.exp(-x))
        self.out = out
        return out
    def backward(self,dout):
        dx = dout * (1.0 - self.out)*self.out
        return dx

## 執行 Affine / Softmax layer